In [1]:
%matplotlib widget

import logging
import time

import cftime
import httpx
import matplotlib.pyplot as plt
import netCDF4
import pandas as pd
import shapely.io
import sqlmodel
from sqlalchemy import (
    bindparam, 
    func
)

from arpav_ppcv import (
    database as db,
    operations,
)
from arpav_ppcv.config import get_settings
from arpav_ppcv.schemas.base import (
    CoverageDataSmoothingStrategy,
    ObservationDataSmoothingStrategy,
    ObservationAggregationType,
    Season,
)
from arpav_ppcv.schemas import coverages
from arpav_ppcv.schemas.coverages import CoverageInternal
from arpav_ppcv.schemas import observations

logging.basicConfig(level=logging.DEBUG)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

settings = get_settings()
session = sqlmodel.Session(db.get_engine(settings))
http_client = httpx.Client()

In [2]:
all_cov_confs = db.collect_all_coverage_configurations(session)

In [3]:
cc = all_cov_confs[0]

In [9]:
cc.possible_values[0].configuration_parameter_value.configuration_parameter.name

'year_period'

In [10]:
[pv for pv in cc.possible_values if pv.configuration_parameter_value.configuration_parameter.name == "climatological_variable"][0]

ConfigurationParameterPossibleValue(configuration_parameter_value_id=UUID('dae44b14-c413-44db-8970-f2236b70028a'), coverage_configuration_id=UUID('268a76ab-4675-4234-a212-15108fa693d7'))

In [12]:
cc.possible_values[0].configuration_parameter_value.name

'MAM'

In [2]:
result = operations.get_forecast_variable_parameters(session)

INFO:arpav_ppcv.operations:coverage configuration 'tas_barometro_climatico' does not have a `aggregation_period`, skipping...
INFO:arpav_ppcv.operations:coverage configuration 'tas_barometro_climatico_lower_uncertainty' does not have a `aggregation_period`, skipping...
INFO:arpav_ppcv.operations:coverage configuration 'tas_barometro_climatico_upper_uncertainty' does not have a `aggregation_period`, skipping...


In [3]:
[k for k in result.keys() if "tas-" in k]

['tas-30yr-anomaly', 'tas-annual-absolute', 'tas-annual-anomaly']

In [4]:
result["tas-30yr-anomaly"].keys()

dict_keys(['climatological_variable', 'aggregation_period', 'measure', 'combinations'])

In [6]:
result["tas-30yr-anomaly"]["climatological_variable"]

ConfigurationParameterValue(id=UUID('e6ff2bf7-67c4-48f9-ad3a-e0737cc2c07a'), display_name_italian='TAS', description_italian='Temperatura media', name='tas', display_name_english='TAS', description_english='Mean temperature', configuration_parameter_id=UUID('12dfb711-9c18-46ac-8fc5-0b902eb48964'))

In [7]:
result["tas-30yr-anomaly"]["aggregation_period"]

ConfigurationParameterValue(id=UUID('b137bb59-a00b-4ed0-a5ea-3ecb4d6c9f7f'), display_name_italian='30 anni', description_italian="I set di dati contengono un'aggregazione di 30 anni", name='30yr', display_name_english='30 Years', description_english='Datasets contain aggregation of 30 years', configuration_parameter_id=UUID('52732ef9-db4c-48af-984f-39b35f3f4fa2'))

In [8]:
result["tas-30yr-anomaly"]["measure"]

ConfigurationParameterValue(id=UUID('dab01644-d1cd-4b31-91ae-34a17b5292f5'), display_name_italian='Anomalia climatologica', description_italian='Rappresenta i valori di anomalia climatologica per la variabile', name='anomaly', display_name_english='Climatological anomaly', description_english='Represents climatological anomaly values for the variable', configuration_parameter_id=UUID('ab4becc0-57db-4716-b8d4-b3ca5eba0ce9'))

In [9]:
result["tas-30yr-anomaly"]["combinations"].keys()

dict_keys(['year_period', 'climatological_model', 'scenario', 'time_window'])

In [10]:
result["tas-30yr-anomaly"]["combinations"]["year_period"]["values"]

[ConfigurationParameterValue(id=UUID('1021b1a8-9e9f-4c90-b483-9211bc17c2e0'), display_name_italian='Primavera', description_italian='Stagione climatologica primaverile (marzo, aprile, maggio)', name='MAM', display_name_english='Spring', description_english='Climatological spring season (March, April, May)', configuration_parameter_id=UUID('1f0a6946-ae9a-48d8-81ef-ef18b14ac767')),
 ConfigurationParameterValue(id=UUID('731991b0-d6fd-48ad-976a-3519ee5e8e52'), display_name_italian='Estate', description_italian='Stagione climatologica estiva (giugno, luglio, agosto)', name='JJA', display_name_english='Summer', description_english='Climatological summer season (June, July, August)', configuration_parameter_id=UUID('1f0a6946-ae9a-48d8-81ef-ef18b14ac767')),
 ConfigurationParameterValue(id=UUID('94b9b78b-e344-4288-a4e8-aa54f83ef63e'), display_name_italian='Autunno', description_italian='Stagione climatologica autunnale (settembre, ottobre, novembre)', name='SON', display_name_english='Autumn', 

In [12]:
result["tas-annual-absolute"]["combinations"]["year_period"]["values"]

[ConfigurationParameterValue(id=UUID('c2834ce6-b5a4-4ace-b569-b9a00a957413'), display_name_italian='Anno', description_italian="L'intero anno", name='year', display_name_english='Year', description_english='Whole year', configuration_parameter_id=UUID('1f0a6946-ae9a-48d8-81ef-ef18b14ac767')),
 ConfigurationParameterValue(id=UUID('1021b1a8-9e9f-4c90-b483-9211bc17c2e0'), display_name_italian='Primavera', description_italian='Stagione climatologica primaverile (marzo, aprile, maggio)', name='MAM', display_name_english='Spring', description_english='Climatological spring season (March, April, May)', configuration_parameter_id=UUID('1f0a6946-ae9a-48d8-81ef-ef18b14ac767')),
 ConfigurationParameterValue(id=UUID('731991b0-d6fd-48ad-976a-3519ee5e8e52'), display_name_italian='Estate', description_italian='Stagione climatologica estiva (giugno, luglio, agosto)', name='JJA', display_name_english='Summer', description_english='Climatological summer season (June, July, August)', configuration_parame